In [13]:
import os
import json
import numpy as np
import pdal
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, Point, mapping

In [2]:
def read_json(json_path):
    try:
        with open(json_path) as js:
            json_obj = json.load(js)
        return json_obj

    except FileNotFoundError:
        print('File not found.')

In [3]:
a = read_json("../pipeline.json")

In [4]:
a

{'pipeline': [{'bounds': '([-10425171.940, -10425161.940], [5164494.710, 5164500.710])',
   'filename': 'https://s3-us-west-2.amazonaws.com/usgs-lidar-public/IA_FullState/ept.json',
   'type': 'readers.ept',
   'tag': 'readdata'},
  {'limits': 'Classification![7:7]',
   'type': 'filters.range',
   'tag': 'nonoise'},
  {'in_srs': 'EPSG:3857',
   'out_srs': 'EPSG:4326',
   'tag': 'reprojectUTM',
   'type': 'filters.reprojection'},
  {'filename': '../data/iowa.csv',
   'tag': 'writerslas',
   'type': 'writers.text'}]}

In [5]:
pipe=pdal.Pipeline(json.dumps(a))
pipe.execute()

21

In [6]:
MINX, MINY, MAXX, MAXY = [-93.759055, 41.925015, -93.766155, 41.935015]
polygon = Polygon(((MINX, MINY), (MINX, MAXY), (MAXX, MAXY), (MAXX, MINY), (MINX, MINY)))

In [7]:
class LidarProcessor:
    
    def __init__(self, public_data_url: str = "https://s3-us-west-2.amazonaws.com/usgs-lidar-public/IA_FullState/ept.json", pipeline_json_path: str="pipeline.json") -> None:
            
        self.pipeline_json = read_json(pipeline_json_path)
        self.public_data_url = public_data_url
        self.input_epsg = 3857
    
        
    def get_polygon_boundaries(polygon: Polygon):
        polygon_df = gpd.GeoDataFrame([polygon], columns=['geometry'])

        polygon_df.set_crs(epsg=4326, inplace=True)
        polygon_df['geometry'] = polygon_df['geometry'].to_crs(epsg=3857)
        minx, miny, maxx, maxy = polygon_df['geometry'][0].bounds

        polygon_input = 'POLYGON(('
        xcords, ycords = polygon_df['geometry'][0].exterior.coords.xy
        for x, y in zip(list(xcords), list(ycords)):
            polygon_input += f'{x} {y}, '
        polygon_input = polygon_input[:-2]
        polygon_input += '))'

        return f"({[minx, maxx]},{[miny,maxy]})", polygon_input
    
    def read_csv(csv_path, missing_values=["n/a", "na", "undefined"]):
        try:
            df = pd.read_csv(csv_path, na_values=missing_values)
            return df

        except FileNotFoundError:
            print('File not found.')
            


In [17]:
df = LidarProcessor.read_csv("../data/iowa.csv")
shape, poly = LidarProcessor.get_polygon_boundaries(polygon)
print(poly)

POLYGON((-10437210.259858532 5149753.664381643, -10437210.259858532 5151249.971344454, -10438000.628243161 5151249.971344454, -10438000.628243161 5149753.664381643, -10437210.259858532 5149753.664381643))


In [20]:
url="https://s3-us-west-2.amazonaws.com/usgs-lidar-public/"

In [31]:
def get_pipeline(region: str, polygon: Polygon):
    path = f"{url}{region}/ept.json"

    a['pipeline'][1]['polygon'] = poly
    pipeline = pdal.Pipeline(json.dumps(a))

    return pipeline


In [28]:
region = "IA_FullState"
pipeline = get_pipeline(region, polygon)
pipeline